<a href="https://colab.research.google.com/github/reckoning-machines/sec_text_nlp/blob/master/r_getter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# why use R here?  
# edgarWebR pulls sections really well
# do i want to find a python library for the same thing?  sure.
# do i want to write a python utils file for the same thing?  not really but we may have to!

#first pull the ticker list with google drive handler lib
!git clone https://gist.github.com/dc7e60aa487430ea704a8cb3f2c5d6a6.git /tmp/colab_util_repo
!mv /tmp/colab_util_repo/colab_util.py colab_util.py 
!rm -r /tmp/colab_util_repo
from colab_util import *
drive_handler = GoogleDriveHandler()

drive_handler.download('test_ticker_list.csv', target_path='test_ticker_list.csv')

#import pandas as pd
#df_tickers = pd.read_csv('test_ticker_list.csv')
#df_tickers = df_tickers[df_tickers['Symbol']=='FB']
#df_tickers.to_csv('test_ticker_list.csv')


Cloning into '/tmp/colab_util_repo'...
remote: Enumerating objects: 40, done.
remote: Total 40 (delta 0), reused 0 (delta 0), pack-reused 40
Unpacking objects: 100% (40/40), done.


In [22]:
# activate R magic
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [23]:
%%R

#R GETTER.  For this example, only one ticker (head,1) is pulled 
#file is saved into local 
#then python code cell pushes file to google sheet
#devtools::install_github("tidyverse/googlesheets4")

#devtools::install_version("xml2", version = "1.2.2", repos = "http://cran.us.r-project.org")
#file creates a set of csv from ticker list which include metadata & text data.


  #devtools::install_packages('trinker/textclean')
  devtools::install_github("mwaldstein/edgarWebR")
  devtools::install_github("r-lib/xml2") #this for edgarWebR 
  devtools::install_github('trinker/textclean')

  library(edgarWebR) #this is an up to date library with an active maintainer.
  library(xml2)
  library(knitr)
  library(dplyr)
  library(purrr)
  library(rvest)
  library(tidyr)
  library(readr)
  #library(textshape)
  #library(lexicon)
  library(textclean)

  #library(log4r) TODO logging file.
  #library(googlesheets4)
  #gs4_deauth()
  #tickers list
  #str_sheet <- "1_xcDVKjR2jqE-w5LqxIWnrDYstpSrE5nFSSY0WXNOVE"
  #df_tickers <- read_sheet(str_sheet)

  #helper functions

  get_filings_links <-function(str_ticker) {
    df_filings <- company_filings(str_ticker, type = "10-", count = 20)
    df_filings <- df_filings[df_filings$type == "10-K" | df_filings$type == "10-Q", ]
    df_filing_infos <- map_df(df_filings$href, filing_information)
    df_filings <- bind_cols(df_filings, df_filing_infos)
    return(head(as_tibble(df_filings),6))
  }




R[write to console]: Skipping install of 'edgarWebR' from a github remote, the SHA1 (e7fa70ea) has not changed since last install.
  Use `force = TRUE` to force installation

R[write to console]: Skipping install of 'xml2' from a github remote, the SHA1 (876759f3) has not changed since last install.
  Use `force = TRUE` to force installation

R[write to console]: Skipping install of 'textclean' from a github remote, the SHA1 (184d7868) has not changed since last install.
  Use `force = TRUE` to force installation



In [24]:
%%R

get_mdna_text <- function(str_href) {
  print(str_href)
  
  #default search
  str_section = 'item 2|item 7'
  str_search = 'discussion'  
  
  df_filing_documents <- filing_documents(str_href)
  str_doc_href <- df_filing_documents[df_filing_documents$type == "10-K" | df_filing_documents$type == "10-Q",]$href
  doc <- parse_filing(str_doc_href)

  #return(doc)
    
  df_txt <- doc[grepl(str_section, doc$item.name, ignore.case = TRUE) & grepl(str_search, doc$item.name, ignore.case = TRUE), ] # only discussion for now
  #if default search fails, use a dictionary attempt
  if (nrow(df_txt) == 0) {
    print('going to backup')
    # paired vector of start and ending text to slice if found

    #JPM and #ECL
    vec_start_end <- c('EXECUTIVE OVERVIEW'='CONSOLIDATED RESULTS OF OPERATIONS',
                       'The following management discussion and analysis'='NON-GAAP FINANCIAL MEASURES')
    
    #this would be case sensitive
    for (start_text in names(vec_start_end)) {
      
      end_text = as.character(vec_start_end[start_text])
      
      print(paste0('trying ',start_text))
      print(paste0('to ',end_text))
      
      i_start = as.integer(which(grepl(start_text, doc$text))) #this should be a loop for each dictionary item
      i_end = as.integer(which(grepl(end_text, doc$text)))
      
      
      print(i_start)
      print(i_end)
      
      #return(i_start)
      
      if (length(i_start) != 0 & length(i_end) != 0) {
        df_txt = doc[i_start:i_end,]
      }
      
    }
    
  }
  #we could do some text preprocessing here.
  
  df_txt <- as_tibble(df_txt) %>%
    #mutate(text = textclean::strip(text)) %>%
    mutate(section = str_search)
  
  return(df_txt)
}

get_section_text <- function(str_href, str_section, str_search) {
  print(str_href)
  
  df_filing_documents <- filing_documents(str_href)
  str_doc_href <- df_filing_documents[df_filing_documents$type == "10-K" | df_filing_documents$type == "10-Q",]$href
  doc <- parse_filing(str_doc_href)
  
  df_txt <- doc[grepl(str_section, doc$item.name, ignore.case = TRUE) & grepl(str_search, doc$item.name, ignore.case = TRUE), ] # only discussion for now
  #we could do some text preprocessing here.
  
  df_txt <- as_tibble(df_txt) %>%
    mutate(text = textclean::strip(text)) %>%
    mutate(section = str_search)
  
  return(df_txt)
}


get_document_text <- function(str_ticker, force = FALSE) { #not using force yet
  start_time <- Sys.time()
  
  print(str_ticker)
  
  str_write_name <- paste0('sec_data_folder/',str_ticker)
  
  print("get filings links ...")
  
  df_filings <- get_filings_links(str_ticker)
  
  print("get section text ...")
  
  df_data <- (df_filings) %>% 
    rowwise() %>%
    mutate(nest_discussion = map(.x = href, .f = get_mdna_text)) %>%
    ungroup() %>%
    group_by(period_date) %>%
    arrange(desc(period_date))
  
  #jenky - find a rowwise application
  a <- df_data %>% 
    select(period_date,filing_date,type,form_name,documents,nest_discussion) %>%
    unnest(nest_discussion)

  print("write to local csv  ...")
  df_data <- a %>%
    as_tibble() %>%
    write_csv(paste0(str_write_name,".csv"))
  
  end_time <- Sys.time()
  
  print(end_time - start_time)
  
  return(df_data)
}

#long run.
df_tickers <- read_csv('test_ticker_list.csv')
dir.create('sec_data_folder', showWarnings = FALSE)

#file creates a set of csv from ticker list which include metadata & text data.
df_tickers <- (df_tickers)

df_data <- map_df(df_tickers$Symbol, get_document_text)
#print(head(df_data))

R[write to console]: Parsed with column specification:
cols(
  X1 = col_double(),
  `GICS Sector` = col_character(),
  Symbol = col_character(),
  mkt_cap = col_double()
)



[1] "GOOG"
[1] "get filings links ..."
[1] "get section text ..."
[1] "https://www.sec.gov/Archives/edgar/data/1652044/000165204420000021/0001652044-20-000021-index.htm"
[1] "https://www.sec.gov/Archives/edgar/data/1652044/000165204420000008/0001652044-20-000008-index.htm"
[1] "https://www.sec.gov/Archives/edgar/data/1652044/000165204419000032/0001652044-19-000032-index.htm"
[1] "https://www.sec.gov/Archives/edgar/data/1652044/000165204419000023/0001652044-19-000023-index.htm"
[1] "https://www.sec.gov/Archives/edgar/data/1652044/000165204419000015/0001652044-19-000015-index.htm"
[1] "https://www.sec.gov/Archives/edgar/data/1652044/000165204419000004/0001652044-19-000004-index.htm"
[1] "write to local csv  ..."
Time difference of 14.80544 secs
[1] "FB"
[1] "get filings links ..."
[1] "get section text ..."
[1] "https://www.sec.gov/Archives/edgar/data/1326801/000132680120000048/0001326801-20-000048-index.htm"
[1] "https://www.sec.gov/Archives/edgar/data/1326801/000132680120000013/0001326

In [26]:
#push files to google drive
import os.path
from os import path
from colab_util import *
drive_handler = GoogleDriveHandler()

sec_folder_id = drive_handler.create_folder('sec_data_folder')

import pandas as pd
df_tickers = pd.read_csv('test_ticker_list.csv')

for i, row in df_tickers.iterrows():
  str_ticker = row['Symbol']
  
  print('working for:'+str_ticker+".csv ...")

  str_to_file = 'sec_data_folder'
  str_from_file = 'sec_data_folder/'+str_ticker+'.csv'

  if path.exists(str_from_file): # a rare example of error handling
    drive_handler.upload(str_from_file, parent_path=str_to_file,overwrite = False)

print("done!")


sec_data_folder already exists
working for:GOOG.csv ...
working for:FB.csv ...
working for:AMZN.csv ...
working for:HD.csv ...
working for:WMT.csv ...
working for:PG.csv ...
working for:XOM.csv ...
working for:CVX.csv ...
working for:JPM.csv ...
working for:BAC.csv ...
working for:JNJ.csv ...
working for:UNH.csv ...
working for:LMT.csv ...
working for:UNP.csv ...
working for:MSFT.csv ...
working for:AAPL.csv ...
working for:LIN.csv ...
working for:ECL.csv ...
working for:AMT.csv ...
working for:CCI.csv ...
working for:NEE.csv ...
working for:D.csv ...
done!
